In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [7]:
##Completar datos e introducir os tres csvs. 
## Previo a todo isto , hai que desaleatorizar nos participants. 
df_1 = pd.read_csv('ethnic_markers_acabado.csv')
df_2 = pd.read_csv('norm_compliance_acabado.csv')
participants = pd.read_csv('Participants.csv')
rounds_1 = int(max(df_1['subsession.round_number'].unique()))
rounds_2 = int(max(df_2['subsession.round_number'].unique()))
num_participants = int(max(df_1['participant.id_in_session']))
#####################

In [8]:
#####Comments
#-Temos que mirar que hacer co participant.code, co participant.id_in_session e co player.id_in_group

#####
def dar_xeito(df_1,df_2,participants,rounds_1,num_participants):
    ### Primeiro, temos que desbotar ós que quedaron inactivos 
    inactives = df_1['participant.id_in_session'][df_1['player.absence']>=4].unique()
    num_participants -= len(inactives)
    for inactive in inactives: 
            df_1.drop(df_1[df_1['participant.id_in_session'] == inactive].index,inplace = True)
            df_2.drop(df_2[df_2['participant.id_in_session'] == inactive].index,inplace = True)
            participants.drop(participants[participants['Player'] == inactive].index,inplace = True)
    ###
    df_1_cols = df_1.columns
    col_1_interest = ['participant.code','participant.id_in_session','subsession.round_number']
    for col in df_1_cols:
        if 'player' in col:
            col_1_interest.append(col)
    df_2_cols =df_2.columns
    col_2_interest = ['participant.code','participant.id_in_session','subsession.round_number']
    for col in df_2_cols:
        if 'player' in col:
            col_2_interest.append(col)
    df_1 = df_1[col_1_interest]
    df_1 = df_1.iloc[:num_participants*rounds_1]
    df_2 = df_2[col_2_interest]
    df_2 = df_2.iloc[:num_participants*rounds_2]
    df_2.fillna(-1,inplace=True)
    participants['Paypal'] = list(df_2['player.paypal'].iloc[len(df_2)-num_participants:])
    participants['Bias'] = list(df_1['player.bias'].iloc[len(df_1)-num_participants:])
    participants['Bias_payoff'] = list(df_1['player.bias_payoff'].iloc[len(df_1)-num_participants:])
    participants['Marker'] = list(df_1['player.marker'].iloc[len(df_1)-num_participants:])
    participants['Comments'] = list(df_2['player.comments'].iloc[len(df_2)-num_participants:])
    del df_1['player.bias']
    del df_1['player.bias_payoff']
    del df_1['player.marker']
    del df_2['player.paypal']
    del df_2['player.payoff']
    del df_2['player.comments']
    
    df_1.columns = [col.split('.')[-1] for col in df_1.columns]
    df_2.columns = [col.split('.')[-1] for col in df_2.columns]
    ########
    players_info = [[0 for i in range(3)] for j in range(num_participants)]

    for i,j in enumerate(df_1['id_in_session'].unique()):
        players_info[i][0] = participants.loc[j-1]
        players_info[i][1] = df_1[df_1['id_in_session'] == j]
        players_info[i][2] = df_2[df_2['id_in_session'] == j]
    
    return players_info


#### Coming from players_info, function to create pagos.txt
def crear_txt(players_info,fecha):
    f = open("pagos_"+fecha+".txt","a")
    for i in range(len(players_info)):
    ###Include conversion rate and participation fee
        pago = round(players_info[i][2]['payoff_total'].iloc[-1]*0.125+2,2)
        pago = str(pago).replace('.',',')
        f.write("{}\t{}\tEUR\n".format(players_info[i][0]['Paypal'], pago))
    f.close()
    return 

def bar_action_marker(players_info):
    ## (igual_0, igual_1, desigual_0,desigual_1)
    results_azul = [[0 for ij in range(4)] for j in range(len(players_info[0][1]))]
    results_amarillo = [[0 for ij in range(4)] for j in range(len(players_info[0][1]))]
    
    for player in players_info:
        for i in range(len(player[1])):
            if player[0]['Marker'] == player[1]['marker_partner'].iloc[i]:
                k = 0
            else:
                k = 2      
            if player[0]['Marker'] == 'azul':
                results_azul[i][k+int(player[1]['acción'].iloc[i])-1] += 1
            elif player[0]['Marker'] == 'amarillo':
                results_amarillo[i][k+int(player[1]['acción'].iloc[i])-1] += 1
        dic_results = {'azul':results_azul,'amarillo':results_amarillo}
    return dic_results



In [9]:
players_info = dar_xeito(df_1,df_2,participants,rounds_1,num_participants)

In [10]:
crear_txt(players_info,'26_02_2021')

In [6]:
def plot_bars(dic_results)####de aqui para abaixo hai que miralo todo
    dic_results = bar_action_marker(players_info)
    final_azul = [0]*4
    flag = [0]*(len(dic_results['azul']))
    for k in range(4):
        l1 = [dic_results['azul'][i][k] for i in range(len(dic_results['azul']))]
        flag = [a+b for a, b in zip(flag, l1)]
        final_azul[k] = flag
    df_azul =pd.DataFrame({'round':range(30),'=,0':final_azul[0],'=,1':final_azul[1],'!=,0':final_azul[2],'!=,1':final_azul[3]})


    final_amarillo = [0]*4
    flag = [0]*(len(dic_results['amarillo']))
    for k in range(4):
        l1 = [dic_results['amarillo'][i][k] for i in range(len(dic_results['amarillo']))]
        flag = [a+b for a, b in zip(flag, l1)]
        final_amarillo[k] = flag
    df_amarillo =pd.DataFrame({'round':range(30),'=,0':final_amarillo[0],'=,1':final_amarillo[1],'!=,0':final_amarillo[2],'!=,1':final_amarillo[3]})        



    sns.set_color_codes("pastel")
    f, ax = plt.subplots(figsize=(12, 8))
    sns.barplot(x='round',y='!=,1',data = df_azul,color = 'b',label = '!=,1')
    sns.barplot(x='round',y='!=,0',data = df_azul,color = 'r',label = '!=,0')
    sns.barplot(x='round',y='=,1',data = df_azul,color = 'c',label = '=,1')
    sns.barplot(x='round',y='=,0',data = df_azul,color = 'g',label = '=,0')
    ax.legend(ncol=4, loc="best", frameon=True)
return

SyntaxError: invalid syntax (<ipython-input-6-e5901a34dfc8>, line 1)

In [ ]:
sns.set_color_codes("pastel")
f, ax = plt.subplots(figsize=(12, 8))
sns.barplot(x='round',y='!=,1',data = df_azul,color = 'b',label = '!=,1')
sns.barplot(x='round',y='!=,0',data = df_azul,color = 'r',label = '!=,0')
sns.barplot(x='round',y='=,1',data = df_azul,color = 'c',label = '=,1')
sns.barplot(x='round',y='=,0',data = df_azul,color = 'g',label = '=,0')
ax.legend(ncol=4, loc="best", frameon=True)


In [ ]:
sns.set_color_codes("pastel")
f, ax = plt.subplots(figsize=(12, 8))
sns.barplot(x='round',y='!=,1',data = df_amarillo,color = 'b',label = '!=,1')
sns.barplot(x='round',y='!=,0',data = df_amarillo,color = 'r',label = '!=,0')
sns.barplot(x='round',y='=,1',data = df_amarillo,color = 'c',label = '=,1')
sns.barplot(x='round',y='=,0',data = df_amarillo,color = 'g',label = '=,0')
ax.legend(ncol=4, loc="best", frameon=True)

In [ ]:
df_total = df_azul+df_amarillo
sns.set_color_codes("pastel")
f, ax = plt.subplots(figsize=(12, 8))
sns.barplot(x='round',y='!=,1',data = df_total,color = 'b',label = '!=,1')
sns.barplot(x='round',y='!=,0',data = df_total,color = 'r',label = '!=,0')
sns.barplot(x='round',y='=,1',data = df_total,color = 'c',label = '=,1')
sns.barplot(x='round',y='=,0',data = df_total,color = 'g',label = '=,0')
ax.legend(ncol=4, loc="best", frameon=True)

In [ ]:
###Cosas - Evolución das coordinacións
def ev_average(players_info,variable):
    rounds = int(max(players_info[0][1].round_number))
    x = [0]*rounds
    var_x = [0]*rounds
    for k in range(rounds):
        x[k] = np.average([players_info[i][1][variable].iloc[k]/(k+1) for i in range(num_participants)])
        var_x[k] = np.var([players_info[i][1][variable].iloc[k]/(k+1) for i in range(num_participants)])
    
    return x,var_x

In [ ]:
for i,j in enumerate(df_1['participant.id_in_session'].unique()):
    participants.iloc[i]
    print(i,j)